In [12]:
from sklearn.externals import joblib

In [13]:
clf = joblib.load('classify_caffe.model') # 분류기 엔진
cate_dict = joblib.load('cate_dict_caffe.dat') # id별 카테고리가 맵핑되어있는 파일
vectorizer = joblib.load('vectorizer_caffe.dat') # 제목들을 숫자 id로 변환해주는 것

In [14]:
cate_id_name_dict = dict(map(lambda (k,v):(v,k),cate_dict.items()))

In [15]:
# initialize size
size = 0
joblib.dump(size,'size.dat')

['size.dat']

In [16]:
# read json results of samples
list_caffe_samples = joblib.load('list_caffe_samples.dat')
'''
res = list_caffe_samples[size]['result']
    if res[0] :
        for each in res[2] :
            name += " " + each[0].replace(" ", "")
'''

'\nres = list_caffe_samples[size][\'result\']\n    if res[0] :\n        for each in res[2] :\n            name += " " + each[0].replace(" ", "")\n'

In [ ]:
# server for 206 samples tests

from bottle import route, run, template,request,get, post


import  time
from threading import  Condition
_CONDITION = Condition()

@route('/classify')
def classify():
    print "classify called"
    img = request.GET.get('img','')
    name = request.GET.get('name', '')
    
    # read size
    size = joblib.load('size.dat')
    print size
    
    # append caffe result
    res = list_caffe_samples[size]['result']
    if res[0] :
        for each in res[2] :
            name += " " + each[0].replace(" ", "")
    
    vec = vectorizer.transform([name])
    p = clf.predict(vec)
    pred = p[0]
    
    # update and store size
    size = size + 1;
    joblib.dump(size,'size.dat')
    
    return {'cate':cate_id_name_dict[pred]}


run(host='0.0.0.0', port=8887)
# http://192.168.99.100:8887/classify?name=%27ad%27&img=%27asd%27

Bottle v0.12.9 server starting up (using WSGIRefServer())...
Listening on http://0.0.0.0:8887/
Hit Ctrl-C to quit.



 * 추후 여기 docker 에서 뭔가 python package 설치할게 있으면 
 * /opt/conda/bin/pip2 install bottle 이런식으로 설치 가능